In [ ]:
import sys
sys.path.append("d:\\SMU\\ml&applns")

import pandas as pd
import numpy as np
import tensorflow as tf
from os import listdir
from os.path import isfile, join
from scipy.stats import skew, kurtosis, entropy

In [ ]:
def get_row_id(row):
    return str(row.frame) + "-" + row.type + "-" + str(row.landmark_index)

def duplicate_vals(pf, values: np.array, iter):
    frames_to_dup = pf.loc[pf.frame.isin(values)].copy()
    frames_to_dup.frame += (frames_to_dup.frame-round(frames_to_dup.frame))*0.001 + 0.01*iter
    
    frames_to_dup.row_id = frames_to_dup.apply(get_row_id, axis=1)

    pf = pd.concat([pf, frames_to_dup], ignore_index=True).sort_values('frame')

    return pf

def remove_vals(pf, values: np.array):
    values = np.sort(values)

    for val in values:
        pf = pf.loc[pf.frame != val]

    return pf


def transform_data(pf, frame_amt_goal, iter=1):

    frame_nums = pf.frame.unique()
    frame_diff = abs(frame_amt_goal - len(frame_nums))
    operation = frame_amt_goal > len(frame_nums)

    values_to_operate = np.array([])

    if frame_diff%2 == 1:
        central_point = frame_nums[int(len(frame_nums)/2)]
        values_to_operate = np.append(values_to_operate, [central_point])
        frame_nums = np.delete(frame_nums, [int(len(frame_nums)/2)])
        frame_diff -= 1

    if frame_diff != 0:
        step_val = len(frame_nums)/frame_diff
        step_val = 1 if step_val < 1 else step_val
        
        loop_cnt = len(frame_nums) if frame_diff > len(frame_nums) else frame_diff
        values_to_operate = np.append(values_to_operate, frame_nums[[int(i*step_val) for i in range(0, loop_cnt)]])
    else:
        loop_cnt = 0

    if operation:
        pf = duplicate_vals(pf, values_to_operate, iter)
    else:
        pf = remove_vals(pf, values_to_operate)

    if frame_diff - loop_cnt != 0:
        pf = transform_data(pf, frame_amt_goal, iter+1)

    return pf

def populate_table(pf, video_data):
    frame_num = 0

    for frame in pf.frame.unique():
        x_vals = list(pf['x'].loc[pf.frame==frame])
        y_vals = list(pf['y'].loc[pf.frame==frame])
        z_vals = list(pf['z'].loc[pf.frame==frame])

        video_data[f'{frame_num}x'] = x_vals
        video_data[f'{frame_num}y'] = y_vals
        video_data[f'{frame_num}z'] = z_vals
        
        frame_num += 1

    return video_data

def create_data_table(pf):
    col_labels = ['type','landmark_index']

    for i in range(len(pf.frame.unique())):
        col_labels.append(f'{i}x')
        col_labels.append(f'{i}y')
        col_labels.append(f'{i}z')

    landmarks = []
    types = []

    for i in pf.type.unique():
        for j in pf.landmark_index.loc[pf.type==i].unique():
            landmarks.append(j)
            types.append(i)

    data = {col: [0.0] * len(types) for col in col_labels}
    data['type'] = types
    data['landmark_index'] = landmarks

    video_data = pd.DataFrame(columns=col_labels, data=data)
    video_data = populate_table(pf, video_data)

    return video_data

def apply_PCA(n_components, x_train, x_test):
    pca = PCA(n_components=n_components)
    pca.fit(x_train)
    return pca.transform(x_train), pca.transform(x_test)

def drop_empty_rows(pf):
    pf = pf.drop(pf.loc[(pf.x == 0) & (pf.y == 0) & (pf.z == 0)].index, axis=0)
    return pf

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.Variable(tf.math.reduce_mean(x, axis=0))
    self.std = tf.Variable(tf.math.reduce_std(x, axis=0)) + 0.001

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

In [ ]:
pf = pd.read_parquet("./asl-kaggle/averaged_by_labels/alligator/20165761.parquet")
pf = pf.fillna(0)
pf = pf.drop(pf.loc[pf.type=="face"].index).reset_index(drop=True)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

types = ['pose', 'left_hand', 'right_hand']

ohe = LabelBinarizer()
ohe.fit(types)
transformed = ohe.transform(pf.type)
ohe_df = pd.DataFrame(transformed)
data = pd.concat([pf, ohe_df], axis=1).drop(['type'], axis=1)

In [ ]:
def hot_encode(encoder, df):
    encoded_data = encoder.transform(df.type)
    encoded_df = pd.DataFrame(encoded_data)
    df = pd.concat([df, encoded_df], axis=1).drop(['type'], axis=1)
    return df

In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import LabelBinarizer

types = ['pose', 'left_hand', 'right_hand']
lblBin = LabelBinarizer().fit(types)

# Initialize a list of folder names containing parquet files
folders = ["alligator", "flower", "kiss", "listen", "orange"]

# Initialize lists for aggregated data and labels
aggregated_files = []
labels = []

# Iterate over the folders in the list
for folder in folders:
    # Update path to focus on content inside folder in the current iteration
    path = "./asl-kaggle/by_labels/"+folder+"/"

    # Fetch all file names in the folder
    parquets = [f for f in listdir(path) if isfile(join(path, f))]

    # Iterate over file names in the list (up to the 50th file name)
    for parquet in parquets[:50]:
        # Update path to focus on the file in the current iteration
        parquet_path = path + parquet
        
        # Read the file at the path and load data to pf
        pf = pd.read_parquet(parquet_path)
        
        # Replace all NaN values with 0
        pf = pf.fillna(0)

        # Remove all rows with the type of face
        pf = pf.drop(pf.loc[pf.type=="face"].index).reset_index(drop=True)
        pf = pf.drop(['row_id'], axis=1)

        pf = hot_encode(lblBin, pf)

        frames_data = pf.groupby('frame').apply(lambda group: group.drop('frame', axis=1).to_numpy().tolist())
        video_data = np.array(frames_data.tolist())

        # Append array to list
        aggregated_files.append(video_data)
        labels.append(folder)


In [ ]:
len(aggregated_files[2])

In [ ]:
aggregated_files[249].shape

In [ ]:
# Find the maximum length of arrays
max_length = max(arr.shape[0] for arr in aggregated_files)

# Pad each array along the first dimension (rows)
padded_data = [np.pad(arr, ((0, max_length - arr.shape[0]), (0, 0), (0, 0)), mode='constant') for arr in aggregated_files]

# Convert the list of padded arrays back to a numpy array
padded_data_array = np.array(padded_data)

In [ ]:
padded_data_array.shape